In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm

In [2]:
from src.MM_estimation import calc_variance_of_default_rate
from src.MM_estimation import estimate_w_factor_loading
from src.ML_estimation import calculate_my_likelihood
from src.ML_estimation import calculate_likelihood_ts
from src.ML_estimation import maximum_likelihood_estimation
from src.monte_carlo import monte_carlo_MLE
from src.sucess_probability import p_g

In [3]:
df = pd.read_csv('SP_historical_PD_data.csv', sep=';')
df["pd_total"] = df["Default rate (%)"] / 100
df["pd_inv"] = df["Investment-grade default rate (%)"] / 100
df["pd_spec"] = df["Speculative-grade default rate (%)"] / 100
# Calculate the number of obligors
df['num_of_inv_grades'] = (df['Investment-grade defaults'] / (df["pd_inv"])).round()
df['num_of_spec_grades'] = (
            df['Speculative-grade defaults'] / (df["pd_spec"])).round().astype(int)
df['num_of_total_grades'] = (df['Total defaults*'] / (df["pd_total"])).round().astype(int)

# Fill-out the missing values in num_of_inv_grades column with the difference between num_of_total_grades and num_of_spec_grades
df['num_of_inv_grades'] = np.where(df['num_of_inv_grades'].isna(), df['num_of_total_grades'] - df['num_of_spec_grades'],
                                   df['num_of_inv_grades']).astype(int)

In [4]:
# MM for speculative grade
MM_params = estimate_w_factor_loading(df["pd_spec"], df["num_of_spec_grades"], initial_guess=0.27)
print("Estimated parameters for speculative grade: ", MM_params)

Estimated parameters for speculative grade:  (0.07602889962521624, 0.040069999999999995)


In [9]:
# ML for speculative grade
ML_params = maximum_likelihood_estimation(df["Speculative-grade defaults"], df["num_of_spec_grades"], norm.pdf, p_g, 0.27, -2, [(0, 1), (-np.inf, np.inf)])
print("Estimated parameters for speculative grade: ", ML_params)

Estimated parameters for speculative grade:  (0.0, -1.78533136062324)


In [11]:
Monte_carlo_params = monte_carlo_MLE(df["Speculative-grade defaults"].sum(), df["num_of_spec_grades"].sum(), p_g, 0.27, -2, [(0, 1), (-np.inf, np.inf)], num_of_simulations=10000)
print("Estimated parameters for speculative grade: ", Monte_carlo_params)

Estimated parameters for speculative grade:  (0.0, -1.7853908061981196)
